Найдите на ютубе интересную лекцию и запустите на ней нейро-копирайтера. Получите аудио-дорожку, на шаге транскрибации переведите текст на английский язык (для этого изучите документацию whisper - https://github.com/openai/whisper).

Сделайте нейро консультанта на базе, который бы отвечал на русском языке, а методичку - на английском.

In [ ]:
!pip -q install yt-dlp -U

In [ ]:
!pip install -q pydub tiktoken openai langchain langchain_community langchain-openai faiss-cpu langchain-text-splitters pytube
from IPython.display import clear_output
clear_output()

In [ ]:
import getpass
import os
import subprocess
import openai
from google.colab import userdata
from pytube import YouTube
from pydub import AudioSegment
from openai import OpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.docstore.document import Document


In [ ]:
# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

## Получение аудио-дорожки из видео по ссылке и загрузка в колаб

In [ ]:
import yt_dlp


# Ссылка на видео YouTube для скачивания
URLS = ['https://youtu.be/W5qZbt5_BtI?si=wAbOg5yIvLzUxeIa']  # указываем ссылку для получения аудио-дорожки из видео

# Придумаем короткое имя для сохраняемого аудио файла на английском языке.
name = 'My_Audio_2'
file_name = f"download_{name}.m4a"

# Значения для получения лучшего качества звука из документации.
ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    'outtmpl': file_name,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)
    if error_code == 0:
        print('Загрузка файла прошла успешно!')
    else:
        print(f'Код ошибки: {error_code}')

print(f"\nАудиофайл загружен, имя файла: {file_name}")

In [ ]:
# выводим путь до скачанного файла
audio_file_path = file_name
audio_file_path

In [ ]:
# Инициализация клиента OpenAI
client = OpenAI()

def transcribe_audio_whisper_chunked(audio_path, file_title, save_folder_path, max_duration=5 * 60 * 1000):  # 5 минут
    """
    Функция для транскрибации аудиофайла на части, чтобы соответствовать ограничениям размера API.
    """

    # Создание папки для сохранения результатов, если она ещё не существует
    os.makedirs(save_folder_path, exist_ok=True)

    # Загрузка аудиофайла
    audio = AudioSegment.from_file(audio_path)

    # Создание временной папки для хранения аудио чанков (фрагментов)
    temp_dir = os.path.join(save_folder_path, "temp_audio_chunks")
    os.makedirs(temp_dir, exist_ok=True)

    # Инициализация переменных для обработки аудио чанков
    current_start_time = 0  # Текущее время начала чанка
    chunk_index = 1         # Индекс текущего чанка
    transcriptions = []     # Список для хранения всех транскрибаций

    # Обработка аудиофайла чанками
    while current_start_time < len(audio):
        # Выделение чанка из аудиофайла
        chunk = audio[current_start_time:current_start_time + max_duration]
        # Формирование имени и пути файла чанка
        chunk_name = f"chunk_{chunk_index}.wav"
        chunk_path = os.path.join(temp_dir, chunk_name)
        # Экспорт чанка в формате wav
        chunk.export(chunk_path, format="wav")

        # Проверка размера файла чанка на соответствие лимиту API
        if os.path.getsize(chunk_path) > 26214400:  # 25 MB
            print(f"Chunk {chunk_index} exceeds the maximum size limit for the API. Trying a smaller duration...")
            max_duration = int(max_duration * 0.9)  # Уменьшение длительности чанка на 10%
            os.remove(chunk_path)  # Удаление чанка, превышающего лимит
            continue

        # Открытие файла чанка для чтения в двоичном режиме
        with open(chunk_path, "rb") as src_file:
            print(f"Transcribing {chunk_name}...")
            try:
                # Запрос на транскрибацию чанка с использованием модели Whisper
                transcript_response = client.audio.translations.create(
                    model="whisper-1",
                    file=src_file
                )
                # Добавление результата транскрибации в список транскрипций
                transcriptions.append(transcript_response.text)
            except openai.BadRequestError as e:
                print(f"An error occurred: {e}")
                break

        # Удаление обработанного файла чанка
        os.remove(chunk_path)
        # Переход к следующему чанку
        current_start_time += max_duration
        chunk_index += 1

    # Удаление временной папки с чанками
    os.rmdir(temp_dir)

    # Сохранение всех транскрипций в один текстовый файл
    result_path = os.path.join(save_folder_path, f"{file_title}.txt")
    with open(result_path, "w") as txt_file:
        txt_file.write("\n".join(transcriptions))

    print(f"Transcription saved to {result_path}")

In [ ]:
# Параметры файла и вызов функции
audio_path = '/content/download_My_Audio_2.m4a'
file_title = 'What_is_CORS_How_to_link_the_frontend_and_backend'
save_folder_path = '/content/transcriptions/'

res = transcribe_audio_whisper_chunked(audio_path, file_title, save_folder_path)

In [ ]:
#@title Разделяем текст на логические блоки с выделением названия раздела:
system = '\u0412\u044B \u0433\u0435\u043D\u0438\u0439 \u0442\u0435\u043A\u0441\u0442\u0430, \u043A\u043E\u043F\u0438\u0440\u0430\u0439\u0442\u0438\u043D\u0433\u0430, \u043F\u0438\u0441\u0430\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0430. \u0412\u0430\u0448\u0430 \u0437\u0430\u0434\u0430\u0447\u0430 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0438 \u0440\u0430\u0437\u0431\u0438\u0442\u044C \u0435\u0433\u043E \u043D\u0430 \u044D\u0442\u0438 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0441\u043E\u0445\u0440\u0430\u043D\u044F\u044F \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442 \u043D\u0430 100%' #@param {type:"string"}
user = '\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, \u0434\u0430\u0432\u0430\u0439\u0442\u0435 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C: \u041F\u043E\u0434\u0443\u043C\u0430\u0439\u0442\u0435, \u043A\u0430\u043A\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0432\u044B \u043C\u043E\u0436\u0435\u0442\u0435 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0438 \u043A\u0430\u043A\u043E\u0435 \u043D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u043F\u043E \u0441\u043C\u044B\u0441\u043B\u0443 \u043C\u043E\u0436\u043D\u043E \u0434\u0430\u0442\u044C \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0414\u0430\u043B\u0435\u0435 \u043D\u0430\u043F\u0438\u0448\u0438\u0442\u0435 \u043E\u0442\u0432\u0435\u0442 \u043F\u043E \u0432\u0441\u0435\u043C\u0443 \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u043C\u0443 \u043E\u0442\u0432\u0435\u0442\u0443 \u0432 \u043F\u043E\u0440\u044F\u0434\u043A\u0435: ## \u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u043F\u043E\u0441\u043B\u0435 \u0447\u0435\u0433\u043E \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442, \u043E\u0442\u043D\u043E\u0441\u044F\u0449\u0438\u0439\u0441\u044F \u043A \u044D\u0442\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0422\u0435\u043A\u0441\u0442:' #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
chunk_size = 7000 #@param {type: "slider", min: 1000, max: 7000, step:500}

In [ ]:
# @title Функции
# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить переполнение текста за границы ячейки вывода и обеспечить его перенос.

from langchain.docstore.document import Document

def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-4o-mini'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')

    if model in ['gpt-4o-mini']:
        num_tokens = 0

        for message in messages:
            num_tokens += 4

            for key, value in message.items():
                num_tokens += len(encoding.encode(value))

                if key == 'name':
                    num_tokens -= 1

        num_tokens += 2
        return num_tokens

    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))

    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')

    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-4o-mini'):

    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    # Вывод количества токенов отключен
    # print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк\n\n')
    answer = completion.choices[0].message.content

    return answer


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток

            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'

        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        print(f'{answer}')  # Выводим ответ

    return processed_text, unprocessed_text

In [ ]:
# @title Запуск
file_path = '/content/transcriptions/What_is_CORS_How_to_link_the_frontend_and_backend.txt'
#file_path = '/content/transcriptions/Графы_Алгоритмы_и_Структуры_Данных_на_Python.txt'
# Вызываем функцию обработки для этого файла
processed_text, unprocessed_text = process_one_file(file_path, system, user)


In [ ]:
# Определяем заголовки, на которые следует разбить текст
headers_to_split_on = [
    ("##", "Header 2")
    ]
# Создаем объект для разбиения текста на секции по заголовкам
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text)

In [ ]:
md_header_splits

In [ ]:
# Возвращаем заголовок из метаданных в текст чанка
# Проходим по каждому документу в списке
for document in md_header_splits:
    # Извлекаем заголовок из метаданных документа
    # заголовок хранится под ключом 'Header 2'
    header = document.metadata['Header 2']

    # Добавляем заголовок к содержимому документа, разделяя точкой и пробелом
    # Если необходимо добавить заголовок именно в начало содержимого, используйте следующий формат
    document.page_content = f'{header}. {document.page_content}'

# После выполнения цикла, md_header_splits будет содержать обновлённые документы

In [ ]:
#@title Обрабатываем каждый чанк, выделяя только суть для методички
system = "\u0412\u044B \u0433\u0435\u043D\u0438\u0439 \u043A\u043E\u043F\u0438\u0440\u0430\u0439\u0442\u0438\u043D\u0433\u0430, \u044D\u043A\u0441\u043F\u0435\u0440\u0442 \u0432 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u0438 \u043D\u0430 \u043F\u0430\u0439\u0442\u043E\u043D \u0438 \u0432 \u0442\u0435\u043C\u0435 \"\u0413\u0440\u0430\u0444\u044B, \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u044B \u0438 \u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u044B \u0434\u0430\u043D\u043D\u044B\u0445\". \u0412\u044B \u043F\u043E\u043B\u0443\u0447\u0430\u0435\u0442\u0435 \u0440\u0430\u0437\u0434\u0435\u043B \u043D\u0435\u043E\u0431\u0440\u0430\u0431\u043E\u0442\u0430\u043D\u043D\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u043F\u043E \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u043E\u0439 \u0442\u0435\u043C\u0435. \u041D\u0443\u0436\u043D\u043E \u0438\u0437 \u044D\u0442\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u0432\u044B\u0434\u0435\u043B\u0438\u0442\u044C \u0441\u0430\u043C\u0443\u044E \u0441\u0443\u0442\u044C, \u0442\u043E\u043B\u044C\u043A\u043E \u0441\u0430\u043C\u043E\u0435 \u0432\u0430\u0436\u043D\u043E\u0435, \u0441\u043E\u0445\u0440\u0430\u043D\u0438\u0432 \u0432\u0441\u0435 \u043D\u0443\u0436\u043D\u044B\u0435 \u043F\u043E\u0434\u0440\u043E\u0431\u043D\u043E\u0441\u0442\u0438 \u0438 \u0434\u0435\u0442\u0430\u043B\u0438, \u043D\u043E \u0443\u0431\u0440\u0430\u0432 \u0432\u0441\u044E \"\u0432\u043E\u0434\u0443\" \u0438 \u0441\u043B\u043E\u0432\u0430 (\u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0435\u043D\u0438\u044F), \u043D\u0435 \u043D\u0435\u0441\u0443\u0449\u0438\u0435 \u0441\u043C\u044B\u0441\u043B\u043E\u0432\u043E\u0439 \u043D\u0430\u0433\u0440\u0443\u0437\u043A\u0438." #@param {type:"string"}
user = "\u0418\u0437 \u0434\u0430\u043D\u043D\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u0432\u044B\u0434\u0435\u043B\u0438 \u0442\u043E\u043B\u044C\u043A\u043E \u0446\u0435\u043D\u043D\u0443\u044E \u0441 \u0442\u043E\u0447\u043A\u0438 \u0437\u0440\u0435\u043D\u0438\u044F \u0442\u0435\u043C\u044B \"\u0433\u0440\u0430\u0444\u044B, \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u044B \u0438 \u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u044B \u0434\u0430\u043D\u043D\u044B\u0445\" \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E. \u0423\u0434\u0430\u043B\u0438 \u0432\u0441\u044E \"\u0432\u043E\u0434\u0443\". \u0412 \u0438\u0442\u043E\u0433\u0435 \u0443 \u0442\u0435\u0431\u044F \u0434\u043E\u043B\u0436\u0435\u043D \u043F\u043E\u043B\u0443\u0447\u0438\u0442\u0441\u044F \u0440\u0430\u0437\u0434\u0435\u043B \u0434\u043B\u044F \u043C\u0435\u0442\u043E\u0434\u0438\u0447\u043A\u0438 \u043F\u043E \u0443\u043A\u0430\u0437\u0430\u043D\u043D\u043E\u0439 \u0442\u0435\u043C\u0435. \u041E\u043F\u0438\u0440\u0430\u0439\u0441\u044F \u0442\u043E\u043B\u044C\u043A\u043E \u043D\u0430 \u0434\u0430\u043D\u043D\u044B\u0439 \u0442\u0435\u0431\u0435 \u0442\u0435\u043A\u0441\u0442, \u043D\u0435 \u043F\u0440\u0438\u0434\u0443\u043C\u044B\u0432\u0430\u0439 \u043D\u0438\u0447\u0435\u0433\u043E \u043E\u0442 \u0441\u0435\u0431\u044F. \u041E\u0442\u0432\u0435\u0442 \u043D\u0443\u0436\u0435\u043D \u0432 \u0444\u043E\u0440\u043C\u0430\u0442\u0435 ## \u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u0438 \u0434\u0430\u043B\u0435\u0435 \u0432\u044B\u0434\u0435\u043B\u0435\u043D\u043D\u0430\u044F \u0442\u043E\u0431\u043E\u0439 \u0446\u0435\u043D\u043D\u0430\u044F \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044F \u0438\u0437 \u0442\u0435\u043A\u0441\u0442\u0430. \u0415\u0441\u043B\u0438 \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u043D\u0435 \u0441\u043E\u0434\u0435\u0440\u0436\u0438\u0442\u0441\u044F \u0446\u0435\u043D\u043D\u043E\u0439 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u0438, \u0442\u043E \u043E\u0441\u0442\u0430\u0432\u044C \u0442\u043E\u043B\u044C\u043A\u043E  \u043D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440: \"## \u0412\u0432\u0435\u0434\u0435\u043D\u0438\u0435\". \u0422\u0435\u043A\u0441\u0442:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}


In [ ]:
def process_documents(documents, system, user, temperature):
    """
    Функция принимает чанки, system, user, temperature для модели.
    Она обрабатывает каждый документ, используя модель GPT, конкатенирует результаты в один текст и сохраняет в файл .txt.
    В итоге мы получаем методичку по лекции.
    """
    processed_text_for_handbook = ""  # Строка для конкатенации обработанного текста

    for document in documents:
        # Получаем ответ от модели
        answer = answer_index(system, user, document.page_content, temperature, model='gpt-4o')
        # Добавляем обработанный текст в общую строку
        processed_text_for_handbook += f"{answer}\n\n"

    # Записываем полученный текст в файл
    with open('processed_documents.txt', 'w', encoding='utf-8') as f:
        f.write(processed_text_for_handbook)

    # Функция возвращает путь к файлу с обработанным текстом
    return 'processed_documents.txt'

# Применение функции
file_path = process_documents(md_header_splits, system, user, temperature)
print(f"Обработанный текст сохранен в файле: {file_path}")

In [ ]:
# Чтение и вывод содержимого методички:
with open(file_path, 'r', encoding='utf-8') as f:
    processed_text = f.read()

print(processed_text)

## Создание нейро-консультанта

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import re

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(md_header_splits, embeddings)

In [ ]:
system_for_NA = """Ты - преподаватель, эксперт по теме ''
                  Твоя задача - ответить студенту на вопрос только на основе представленных тебе документов, не добавляя ничего от себя. А также ты должен отвечать на русском языке"""

In [ ]:
def answer_neuro_assist(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=3)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system_for_NA},
        {"role": "user", "content": f"Ответь на вопрос студента. Не упоминай отрывки документов с информацией для ответа студенту в ответе. Ответ должен быть на русском языке. Документ с информацией для ответа студенту: {message_content}\n\nВопрос студента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [ ]:
topic="Что такое CORS?"
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans